In [ ]:
from object.class_file import BatchConfig
from pathlib import Path
from object.class_file import StrategyParams
from backtesting.global_loop import run_global_ranking_walkforward

In [ ]:
PROJECT_PATH = Path(__file__).resolve().parents[0]
BASE_DATA_PATH = PROJECT_PATH / "data" / "raw"
SCANNER_DATA_PATH = BASE_DATA_PATH / "d1"
DATA_PATH = PROJECT_PATH / "data" / "raw" / "d1"

In [ ]:
SELECTED_UNIVERSE = "sweden"

In [ ]:
cfg = BatchConfig(
        data_path = PROJECT_PATH / "data" / "raw" / "d1",
        monthly_universe_path = PROJECT_PATH / "data" / "universe" / f"{SELECTED_UNIVERSE}.parquet",
        out_dir = PROJECT_PATH / "data" / "backtests" / "global_ranking",
        universe_name="GLOBAL",
        timeframe="Daily",
        warmup_extra=50,
        equal_weight=True,
    )

In [ ]:
z_entry = 2
z_exit = 4
z_stop = 0
z_window = 60
wf_train = 120
fees = 0.0002
beta_mode = "monthly"
N = 20
K = 5

params = StrategyParams(
        z_entry=float(z_entry),
        z_exit=float(z_exit),
        z_stop=float(z_stop),
        z_window=int(z_window),
        wf_train=int(wf_train),
        wf_test=0,
        fees=float(fees),
        beta_mode=str(beta_mode),
        top_n_candidates= int(N),
        max_positions= int(K)
)

In [ ]:
res = run_global_ranking_walkforward(
    cfg=cfg,
    params=params,
    universes=list(universes),
)